# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [2]:
import pickle
import re
import pandas as pd
import numpy as np 
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.linear_model import LogisticRegression

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /home/daniel/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/daniel/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/daniel/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# load data from database
engine = create_engine('sqlite:///../data/disaster_project.db')
df = pd.read_sql_table('messages_categories', engine)

features = ['message', 'genre']

X = df[features]
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [7]:
def tokenizer(text: str):
    """Normalize, tokenize and lemmatize text.

    Args:
        text: str. A body of text.

    Returns:
        tokens: list(str). A list of normalised and lemmatized words.
    """
    # Replace url with 'website'.
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    found_urls = re.findall(url_regex, text)
    for url in found_urls:
        text = text.replace(url, "website")

    # Convert to lower case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())

    # Lemmatize and remove stop words.
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words('english')

    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(t) for t in tokens if t not in stop_words]
    return tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [8]:
lr_pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenizer)),
    ('clf', MultiOutputClassifier(LogisticRegression()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [10]:
# Remove the "child_alone" since all labels are 0
# LogisticRegression has a bug that prevents training on one class
columns = Y.columns.tolist()
columns.remove('child_alone') 

X_train, X_test, y_train, y_test = train_test_split(X.message, Y[columns], random_state=1)

model = lr_pipeline
%time model.fit(X_train, y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
def generate_classification_report(y_test: pd.DataFrame, y_pred: pd.DataFrame):
    """Create a classification report for each column including:
    accuracy, precision, recall and f1 score. 

    Args:
        y_test: pandas.DataFrame The test dataset, values of (0,1) for each class.
        y_pred: pandas.DataFrame Predictions, values of (0,1) for each class.
    Returns:
        df: pandas.DataFrame. A dataframe of metrics for each label.
    """    
    metrics = []

    for col in y_test.columns:
        accuracy = accuracy_score(y_test[col], y_pred[col])
        precision = precision_score(y_test[col], y_pred[col], average='macro')
        recall = recall_score(y_test[col], y_pred[col], average='macro')
        f1 = f1_score(y_test[col], y_pred[col], average='macro')

        metrics.append([accuracy, precision, recall, f1])

    return pd.DataFrame(
        data=metrics,
        index=y_test.columns,
        columns=['accuracy', 'precision', 'recall', 'f1']
        ).round(2)

def create_baseline_clf_report(y_test: pd.DataFrame, y_train: pd.DataFrame):
    """Create a baseline classification performance report
    by predicting the majority label (0 or 1) from the training
    dataset for each category.

    Args:
        y_test: pandas.DataFrame The test dataset, values of (0,1) for each class.
    Returns:
        df: pandas.DataFrame. A dataframe of metrics from predicting the majority label
        for each class.
    """    
    baseline_pred = pd.DataFrame(np.zeros(y_test.shape), columns=y_test.columns)
    summary_df = y_train.sum() / y_train.shape[0] 
    majority_classes = summary_df[summary_df > 0.5].index.tolist() 
    for label in majority_classes:
        baseline_pred[label] = 1
    return generate_classification_report(y_test, baseline_pred)

In [7]:
%time y_pred = model.predict(X_test)  
y_pred = pd.DataFrame(y_pred, columns=y_test.columns)

clf_report = generate_classification_report(y_test=y_test, y_pred=y_pred)
baseline_clf_report = create_baseline_clf_report(y_test, y_train)

knn_clf_report = clf_report.merge(baseline_clf_report, left_index=True, right_index=True, suffixes=('', '_bp'))  

CPU times: user 4.85 s, sys: 188 ms, total: 5.04 s
Wall time: 5.09 s


### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
parameters = {
    'clf__estimator__penalty': ['l2', None],
    'clf__estimator__C': [0.1, 1, 10]
}

cv = GridSearchCV(lr_pipeline, param_grid=parameters)
cv.fit(X_train, y_train)

GridSearchCV(cv=None, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                                       

In [18]:
cv.best_params_
# {'clf__estimator__C': 1, 'clf__estimator__penalty': 'l2'}

{'clf__estimator__C': 1, 'clf__estimator__penalty': 'l2'}

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
y_pred = pd.DataFrame(
    cv.predict(X_test),
    columns=y_test.columns
)
lr_clf_report = generate_classification_report(y_test, y_pred)
lr_clf_report

,accuracy,precision,recall,f1
related,0.83,0.53,0.47,0.49
request,0.90,0.86,0.75,0.79
offer,1.00,0.50,0.50,0.50
aid_related,0.78,0.78,0.77,0.77
medical_help,0.92,0.79,0.58,0.62
medical_products,0.95,0.84,0.58,0.61
search_and_rescue,0.97,0.79,0.52,0.53
security,0.98,0.49,0.50,0.50
military,0.97,0.77,0.56,0.60
water,0.96,0.88,0.75,0.79


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

The focus of this project is to build an end-to-end pipeline and web application. We will test two more models and select the best model.

In [22]:
bnb_pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenizer)),
    ('clf', MultiOutputClassifier(BernoulliNB()))
])

knn_pipeline = Pipeline([
    ('vect', TfidfVectorizer(tokenizer=tokenizer)),
    ('clf', MultiOutputClassifier(KNeighborsClassifier(), n_jobs=2))
])

results = []
for pipeline in [bnb_pipeline, knn_pipeline]:
    %time pipeline.fit(X_train, y_train)
    %time y_pred = pipeline.predict(X_test)
    y_pred = pd.DataFrame(y_pred, columns=y_test.columns)
    clf_report = generate_classification_report(y_test=y_test, y_pred=y_pred)
    results.append(clf_report)

CPU times: user 14.8 s, sys: 531 ms, total: 15.3 s
Wall time: 15.4 s
CPU times: user 4.14 s, sys: 152 ms, total: 4.29 s
Wall time: 4.3 s
CPU times: user 12.2 s, sys: 569 ms, total: 12.7 s
Wall time: 13.8 s
CPU times: user 4.42 s, sys: 438 ms, total: 4.86 s
Wall time: 2min 5s


,accuracy,precision,recall,f1
related,0.82,0.51,0.48,0.49
request,0.89,0.83,0.77,0.79
offer,0.99,0.50,0.50,0.50
aid_related,0.74,0.74,0.72,0.73
medical_help,0.91,0.69,0.55,0.57
medical_products,0.94,0.68,0.54,0.55
search_and_rescue,0.97,0.59,0.51,0.52
security,0.98,0.59,0.52,0.53
military,0.97,0.67,0.54,0.56
water,0.93,0.68,0.53,0.55


,accuracy,precision,recall,f1
related,0.77,0.64,0.50,0.53
request,0.86,0.83,0.64,0.67
offer,1.00,0.50,0.50,0.50
aid_related,0.65,0.69,0.59,0.56
medical_help,0.92,0.81,0.52,0.51
medical_products,0.95,0.89,0.51,0.51
search_and_rescue,0.97,0.74,0.51,0.51
security,0.98,0.49,0.50,0.50
military,0.97,0.86,0.52,0.52
water,0.94,0.87,0.55,0.57


In [24]:
display(results[0])
display(results[1])

,accuracy,precision,recall,f1
related,0.82,0.51,0.48,0.49
request,0.89,0.83,0.77,0.79
offer,0.99,0.50,0.50,0.50
aid_related,0.74,0.74,0.72,0.73
medical_help,0.91,0.69,0.55,0.57
medical_products,0.94,0.68,0.54,0.55
search_and_rescue,0.97,0.59,0.51,0.52
security,0.98,0.59,0.52,0.53
military,0.97,0.67,0.54,0.56
water,0.93,0.68,0.53,0.55


,accuracy,precision,recall,f1
related,0.77,0.64,0.50,0.53
request,0.86,0.83,0.64,0.67
offer,1.00,0.50,0.50,0.50
aid_related,0.65,0.69,0.59,0.56
medical_help,0.92,0.81,0.52,0.51
medical_products,0.95,0.89,0.51,0.51
search_and_rescue,0.97,0.74,0.51,0.51
security,0.98,0.49,0.50,0.50
military,0.97,0.86,0.52,0.52
water,0.94,0.87,0.55,0.57


### 9. Export your model as a pickle file

In [15]:
# Fit on all data
model = lr_pipeline
model.fit(X.message, Y[columns])

with open('model.pkl', 'wb') as fh:
    pickle.dump(model, fh)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.